In [2]:
import polars as pl
import glob
from pathlib import Path
from tqdm import tqdm

base_dir = Path('data/raw_data')
cats_tree = glob.glob(str(base_dir / 'ml_ozon_recsys_categories_tree/*.parquet'))[0]
orders_files = glob.glob(str(base_dir / 'ml_ozon_recsys_orders_data/*.parquet'))
tracker_files = glob.glob(str(base_dir /'ml_ozon_recsys_tracker_data/*.parquet'))
items_files = glob.glob(str(base_dir /'ml_ozon_recsys_items_data/*.parquet'))

print(f"Найдено файлов заказов: {len(orders_files)}")
print(f"Найдено файлов взаимодействий: {len(tracker_files)}")
print(f"Найдено файлов товаров: {len(items_files)}")


categories_df = pl.read_parquet(cats_tree)
print(f"Загружено категорий: {len(categories_df):,}")

orders_data = []
for file_path in tqdm(orders_files, desc="Загрузка заказов"):
    df = pl.read_parquet(file_path)
    orders_data.append(df)
orders_df = pl.concat(orders_data)
print(f"Загружено заказов: {len(orders_df):,}")

tracker_data = []
for file_path in tqdm(tracker_files[:20], desc="Загрузка взаимодействий"):
    df = pl.read_parquet(file_path)
    tracker_data.append(df)
tracker_df = pl.concat(tracker_data)
print(f"Загружено взаимодействий: {len(tracker_df):,}")

items_data = []
for file_path in tqdm(items_files[:10], desc="Загрузка товаров"):
    df = pl.read_parquet(file_path)
    items_data.append(df)
items_df = pl.concat(items_data)
print(f"Загружено товаров: {len(items_df):,}")

Найдено файлов заказов: 5
Найдено файлов взаимодействий: 200
Найдено файлов товаров: 100
Загружено категорий: 7,012


Загрузка заказов: 100%|██████████| 5/5 [00:00<00:00,  6.17it/s]


Загружено заказов: 18,887,122


Загрузка взаимодействий: 100%|██████████| 20/20 [00:05<00:00,  3.50it/s]


Загружено взаимодействий: 164,241,333


Загрузка товаров: 100%|██████████| 10/10 [00:05<00:00,  1.82it/s]

Загружено товаров: 644,381


In [3]:
trash_data = []
for file_path in tqdm(tracker_files, desc="Загрузка взаимодействий"):
    df = pl.read_parquet(file_path)
    trash_df = df.filter(pl.col("timestamp").dt.epoch("s") > 1751414398)
    trash_data.append(trash_df)
total_trash_df = pl.concat(trash_data)
print(f"Загружено взаимодействий: {len(total_trash_df):,}")

Загрузка взаимодействий: 100%|██████████| 200/200 [00:46<00:00,  4.33it/s]

Загружено взаимодействий: 2,363


In [22]:
categories_df

catalogid,catalogpath,ids
i64,list[struct[3]],list[i64]
15500,"[{15500,""Электроника"",""Электроника""}, {-1,""Общий каталог"",""Общий каталог""}]","[15500, -1]"
15501,"[{15501,""Телефоны и смарт-часы"",""Телефоны и смарт-часы""}, {15500,""Электроника"",""Электроника""}, {-1,""Общий каталог"",""Общий каталог""}]","[15501, 15500, -1]"
15502,"[{15502,""Смартфоны"",""Смартфоны""}, {15501,""Телефоны и смарт-часы"",""Телефоны и смарт-часы""}, … {-1,""Общий каталог"",""Общий каталог""}]","[15502, 15501, … -1]"
15511,"[{15511,""Аксессуары для смартфонов и телефонов"",""Аксессуары для смартфонов и телефонов""}, {15501,""Телефоны и смарт-часы"",""Телефоны и смарт-часы""}, … {-1,""Общий каталог"",""Общий каталог""}]","[15511, 15501, … -1]"
15892,"[{15892,""Чехлы"",""Чехлы для смартфонов""}, {15511,""Аксессуары для смартфонов и телефонов"",""Аксессуары для смартфонов и телефонов""}, … {-1,""Общий каталог"",""Общий каталог""}]","[15892, 15511, … -1]"
…,…,…
39405,"[{39405,""Екатеринбург"",""Отели Екатеринбурга""}, {55455,""Отели"",""Отели""}, … {-1,""Общий каталог"",""Общий каталог""}]","[39405, 55455, … -1]"
39406,"[{39406,""Нижний Новгород"",""Отели Нижнего Новгорода""}, {55455,""Отели"",""Отели""}, … {-1,""Общий каталог"",""Общий каталог""}]","[39406, 55455, … -1]"
39398,"[{39398,""Экскурсии"",""Экскурсии""}, {55055,""Билеты, отели, туры"",""Билеты, отели, туры""}, {-1,""Общий каталог"",""Общий каталог""}]","[39398, 55055, -1]"


In [ ]:
tracker_df.filter(pl.col("timestamp").dt.epoch("s") > 1751414398)

item_id,user_id,timestamp,action_type,action_widget,date
i32,i32,datetime[ns],str,str,str
30760,1889010,2025-01-10 20:47:00,"""page_view""","""catalog.searchResultsV2""","""2025-01-10"""
30760,3017520,2025-02-10 11:19:49,"""page_view""","""catalog.searchResultsV2""","""2025-02-10"""
30760,2476291,2025-05-02 13:48:47,"""page_view""","""catalog.searchResultsV2""","""2025-05-02"""
30760,89601,2025-06-30 16:18:47,"""page_view""","""shelf.analogLookSimilar""","""2025-06-30"""
30760,3358921,2025-02-08 11:46:43,"""page_view""","""catalog.searchResultsV2""","""2025-02-08"""
…,…,…,…,…,…
338891677,1351550,2025-05-16 17:07:38,"""view_description""","""pdp.textDescription""","""2025-05-16"""
338891677,1351550,2025-05-16 17:07:38,"""view_description""","""pdp.textDescription""","""2025-05-16"""
338891677,1351550,2025-05-16 17:07:27,"""page_view""","""shelf.analogShelfPersonal""","""2025-05-16"""


In [25]:
tracker_df["timestamp"].dt.epoch("s").min(), tracker_df["timestamp"].dt.epoch("s").max()

(1514836475, 1801702159)

In [26]:
tracker_df["timestamp"].min(), tracker_df["timestamp"].max()

(datetime.datetime(2018, 1, 1, 19, 54, 35),
 datetime.datetime(2027, 2, 4, 0, 49, 19))

In [19]:
orders_df["created_timestamp"].dt.epoch("s").min(), orders_df["created_timestamp"].dt.epoch("s").max()

(1735689601, 1751414398)

In [23]:
orders_df["created_timestamp"].min(), orders_df["created_timestamp"].max()


(datetime.datetime(2025, 1, 1, 0, 0, 1, 406000),
 datetime.datetime(2025, 7, 1, 23, 59, 58, 283000))

In [10]:
df = tracker_df.with_columns(pl.col("timestamp").dt.epoch("s").alias("timestamp"))
# tracker_df['timestamp'].max()

In [15]:
tracker_df["timestamp"].dt.epoch("s")

timestamp
i64
1736542020
1739186389
1746193727
1751300327
1739015203
…
1747415258
1747415258
1747415247


In [13]:
ts_s = pl.col("timestamp").dt.epoch("s") 
ts_s

<Expr ['col("timestamp").map()'] at 0x151A0D3A0>

In [5]:
orders_df['last_status'].value_counts()

last_status,count
str,u32
"""proccesed_orders""",960296
"""canceled_orders""",7990577
"""delivered_orders""",9936249


In [8]:
tracker_df['action_type'].value_counts()

action_type,count
str,u32
"""unfavorite""",1982175
"""remove""",3712297
"""favorite""",5199691
"""to_cart""",6096416
"""view_description""",22111951
"""review_view""",7593543
"""page_view""",117545260


In [9]:
tracker_df

item_id,user_id,timestamp,action_type,action_widget,date
i32,i32,datetime[ns],str,str,str
30760,1889010,2025-01-10 20:47:00,"""page_view""","""catalog.searchResultsV2""","""2025-01-10"""
30760,3017520,2025-02-10 11:19:49,"""page_view""","""catalog.searchResultsV2""","""2025-02-10"""
30760,2476291,2025-05-02 13:48:47,"""page_view""","""catalog.searchResultsV2""","""2025-05-02"""
30760,89601,2025-06-30 16:18:47,"""page_view""","""shelf.analogLookSimilar""","""2025-06-30"""
30760,3358921,2025-02-08 11:46:43,"""page_view""","""catalog.searchResultsV2""","""2025-02-08"""
…,…,…,…,…,…
338891677,1351550,2025-05-16 17:07:38,"""view_description""","""pdp.textDescription""","""2025-05-16"""
338891677,1351550,2025-05-16 17:07:38,"""view_description""","""pdp.textDescription""","""2025-05-16"""
338891677,1351550,2025-05-16 17:07:27,"""page_view""","""shelf.analogShelfPersonal""","""2025-05-16"""
